## Capstone 2. Predicting Circular Dichorism Spectrum Features from Amino Acid Sequence
### Feature Engineering
Data is coming from the Protein Circular Dichroism Data Bank (https://pcddb.cryst.bbk.ac.uk/). Citation information: The PCDDB (protein circular dichroism data bank): A bioinformatics resource for protein characterisations and methods development. Ramalli SG, Miles AJ, Janes RW, Wallace BA., J Mol Biol (2022)

In [1]:
# import packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from aaindex import aaindex1
from sklearn.model_selection import train_test_split
from scipy.signal import find_peaks

In [2]:
# Unpickle clean(ish) data
dfOG = pd.read_pickle("C:/Users/dkoul/OneDrive/Documents/Springboard/Capstone 2/pcd_df.pkl")
dfOG.head()

,Info,CD,Calibration
CD0000001000,"{'PCDDBID': 'CD0000001000', 'FILE EXTRACTED': ...",Wavelength Final HT Smoothed...,Wavelength Calibration Spectrum 0 ...
CD0000002000,"{'PCDDBID': 'CD0000002000', 'FILE EXTRACTED': ...",Wavelength Final HT Smoothed ...,Wavelength Calibration Spectrum 0 ...
CD0000003000,"{'PCDDBID': 'CD0000003000', 'FILE EXTRACTED': ...",Wavelength Final HT Smoothed Av...,Wavelength Calibration Spectrum 0 ...
CD0000004000,"{'PCDDBID': 'CD0000004000', 'FILE EXTRACTED': ...",Wavelength Final HT Smoothed Av...,Wavelength Calibration Spectrum 0 ...
CD0000005000,"{'PCDDBID': 'CD0000005000', 'FILE EXTRACTED': ...",Wavelength Final HT Smoothed...,Wavelength Calibration Spectrum 0 ...


#### Simplify the above dataframe to only the relevant fields

- the PCDDB ids (as index)
- amino acid sequence
- wavelengths *(select only 190 - 250, then we can disregard this in model and avoid complicating algorithms with experimental settings/buffer interactions/etc.?)*
- processed/final cd spectrum *(all converted to $\Delta$ $\epsilon$)*

In [21]:
need_tlc = [72, 74, 98, 121, 482, 508] # indices of values where the metadata fields are jumbled up, identified during EDA

# PCDDB ID
ids_all = dfOG.index
tlc_ids = [ids_all[i] for i in need_tlc]
ids = [i for i in ids_all if i not in tlc_ids]

# Breakdown of how to access all the information

wl = range(190, 251)

def extract_data(id):
    units = dfOG.loc[id, 'Info']['Dichroism Units of Processed Data']
    CD = dfOG.loc[id, 'CD']
    if units == 'Delta Epsilon':
        spectrum = CD[CD['Wavelength'].isin(wl)]['Final']
    elif units == 'Mean Residue Ellipticity':
        spectrum = CD[CD['Wavelength'].isin(wl)]['Final']/3298
        
    seq = dfOG.loc[id, 'Info']['Sequence']
    
    return spectrum, seq

# lists to populate
seqs = [] # for amino acid sequences
delta_epsilon = [] # spectra for wavelengths 190 - 250
errors = [] # for anything that didn't work that I need to look at individually

for id in ids:
    try:           
        spectrum, seq = extract_data(id)
        delta_epsilon.append(spectrum.reset_index(drop = True))       
        seqs.append(seq)
    except:
        errors.append(id)

# Combine lists into dataframe
ids_tlc = tlc_ids + errors
ids_notlc = [i for i in ids if i not in ids_tlc]

df_clean1 = pd.DataFrame({'PCDDB ID': ids_notlc, 'Sequence': seqs, 'CD': delta_epsilon})

# Remove any rows that don't have values for the full wavelength range we're looking at
df_clean1['Spectrum Points'] = [len(x) for x in df_clean1['CD']]
df_clean2 = df_clean1[df_clean1['Spectrum Points'] == 61]
df_clean = df_clean2.drop('Spectrum Points', axis = 1).reset_index(drop = True)

df_clean.head()

,PCDDB ID,Sequence,CD
0,CD0000001000,PHSHPALTPE QKKELSDIAH RIVAPGKGIL AADESTGSIA KR...,0 -0.095282 1 -0.150240 2 -0.19838...
1,CD0000002000,RTPEMPVLEN RAAQGDITAP GGARRLTGDQ TAALRDSLSD KP...,0 -0.319511 1 -0.332366 2 -0.282746 3...
2,CD0000003000,ANLNGTLMQY FEWYMPNDGQ HWKRLQNDSA YLAEHGITAV WI...,0 -0.007964 1 -0.073311 2 -0.10272...
3,CD0000004000,IVCHTTATSP ISAVTCPPGE NLCYRKMWCD VFCSSRGKVV EL...,0 0.106316 1 0.125327 2 0.114049 3...
4,CD0000005000,CGVPAIQPVL SGLIVNGEEA VPGSWPWQVS LQDKTGFHFC GG...,0 -0.023542 1 -0.033641 2 -0.039037 3...


#### Explanatory variable processing using AAIndex

The primary amino acid sequences cannot be used in machine learning algorithms as they are. So, we will use AAIndex to convert the amino acid sequences to numeric vectors based on some physiochemical property, which can then be reduced to a single number as either a mean or sum.

https://www.genome.jp/aaindex/
Access via DBGET, explained here: https://www.genome.jp/en/about_dbget.html
Accessed here via Python package: https://pypi.org/project/aaindex/

AAIndex has a LOT of potential amino acid indices. The subset that will be explored here comprises the size, partition coefficient, hydrophobicity, flexibility, chemical shift, helical propensity, and $/phi$ angle. These were chosen as a diverse subset of properties that would be expected to impact secondary structure. The references to the specific indices chosen are specified below in the comment next to each property.

In [23]:
size = aaindex1['BIGC670101'].values # Residue volume (Bigelow, 1967)
partition_coeff = aaindex1['GARJ730101'].values # Partition coefficient (Garel et al., 1973)
hydrophobicity = aaindex1['ARGP820101'].values # Hydrophobicity index (Argos et al., 1982)
flexibility = aaindex1['BHAR880101'].values # Average flexibility indices (Bhaskaran-Ponnuswamy, 1988)
chemical_shifts = aaindex1['ANDN920101'].values # alpha-CH chemical shifts (Andersen et al., 1992)
K_helixcoil = aaindex1['FINA770101'].values #Helix-coil equilibrium constant (Finkelstein-Ptitsyn, 1977)
phi = aaindex1['LEVM760104'].values # Side chain torsion angle phi(AAAR) (Levitt, 1976)

properties = {'size': size, 'partition_coeff': partition_coeff, 'hydrophobicity': hydrophobicity, \
              'flexibility': flexibility, 'chemical_shifts': chemical_shifts, 'K_helixcoil': K_helixcoil, 'phi': phi}

def seq_vectorization(property, df, i):
    '''Convert amino acid sequence to numerical vector using AAIndex for property specified.'''
    seq = df['Sequence'][i]
    num_seq = pd.Series(map(property.get, list(seq)))
    return(num_seq.dropna())

In [24]:
# Iterate over dataframe rows to generate numerical vectors for each amino sequence for all off the chosen properties
vect_dict = {'size': [], 'partition_coeff': [], 'hydrophobicity': [], 'flexibility': [], \
             'chemical_shifts': [], 'K_helixcoil': [], 'phi': []}

for i in range(len(df_clean)):
   for prop in properties:
        vect_dict[prop].append(seq_vectorization(properties[prop], df_clean, i))

df_propvect = df_clean.join(pd.DataFrame(vect_dict))

In [29]:
# Determine mean and sum for each numeric vector representing a sequence

df_propvectcols = df_propvect[properties.keys()]

means = {}

for p in properties.keys():
    means[p] = np.zeros(len(df_propvectcols))
    for idx, data in enumerate(df_propvectcols[p]):
        means[p][idx] = np.mean(data)

sums = {}

for p in properties.keys():
    sums[p] = np.zeros(len(df_propvectcols))
    for idx, data in enumerate(df_propvectcols[p]):
        sums[p][idx] = np.sum(data)

aa_means = pd.DataFrame(means)
aa_means['PCDDB ID'] = df_propvect['PCDDB ID']
aa_sums = pd.DataFrame(sums)
aa_sums['PCDDB ID'] = df_propvect['PCDDB ID']

#### Target variable processing: extracting general features from CD spectra

Reference: Greenfield NJ "Using circular dichrosim spectra to estimate protein secondary structure" Nature Protocols 2006 (doi:10.1038/nprot.2006.202) <p>

- $\alpha$-helices are expressed in negative bands at 222 nm and 208 nm and a positive band at 193 nm
- $\beta$-sheets (antiparallel) are expressed as negative bands 218 nm and positive band at 195 nm
- disordered proteins have very low ellipticity above 210 nm and negative bands near 195 nm
<p>
So we will process the CD data with the following filters to generate three binary target features:<br>
As a first pass, we will say that the criteria will be:
- minimum between 217 - 219 => $\beta$
- minimum between 220 - 224 => $\alpha$
- negative value at 195 => random coil

In [30]:
''' Correcting erroneous entries where some of the CD entries have wavelengths as part of the CD fields 
and are formatted as strings. '''

CD_corr = np.zeros((len(df_propvect['CD']), 61))

for idx, val in enumerate(df_propvect['CD'].values):
    try:
        CD_corr[idx] = np.array(val).astype('float32')
    except ValueError:
        val = [s.split()[0] for s in val]
        CD_corr[idx] = np.array(val).astype('float32')
        
target_df_num = pd.DataFrame(CD_corr, columns = wl)

In [31]:
# find wavelengths at which spectrums has minima
wl_list = list(target_df_num.columns)
minima_at = []
for idx, data in target_df_num.iterrows():
    min_i, prop = find_peaks(-data)
    min_wl = [wl_list[i] for i in min_i]
    minima_at.append(min_wl)

In [33]:
# Evaluate criteria for alpha helix & beta sheet
alpha = np.zeros(len(target_df_num))
beta = np.zeros(len(target_df_num))

for idx, x in enumerate(minima_at):
    for y in x:
        if 217 <= y <=219:
            beta[idx] = True
        if 220 <= y <= 224:
            alpha[idx] = True

# Evaluate criteria for random coil
random_coil = [(x < 0) for x in target_df_num[195]]

# Combine into dataframe
target_df_cat = pd.DataFrame({'alpha': [bool(x) for x in alpha], 'beta': [bool(x) for x in beta], 'random_coil': random_coil})

# add PCDDB ID for reference
target_df_cat['PCDDB ID'] = df_propvect['PCDDB ID']

target_df_cat.head()

,alpha,beta,random_coil,PCDDB ID
0,False,True,True,CD0000001000
1,True,False,True,CD0000002000
2,True,False,True,CD0000003000
3,False,False,False,CD0000004000
4,False,False,True,CD0000005000


### Conclusions
We now have our amino acid sequences encoded as either means ([aa_means]) or sums ([aa_sums]) of their relative index based on six physiochemical properties, hopefully capturing some emergent property or properties of the sequence in one number. Our target variables are three binary categorical features summarizing features of the CD spectra most indicative of secondary structure components (in [target_df_cat]). 
<p>
During EDA, three different modeling algorithms showed the most promise:
    - RandomForestClassifier
    - AdaBoostClassifier
    - SVC (Support Vector Classification)n)
</
These dataframes will serve as inputs and outputs for the algorithms to either determine an approach to predict broad secondary structural elements from amino acid sequences and/or lend insights into which of the physiochemical properties evaluated are the most deterministic of one or all of the secondary structure elements explored.p>